In [0]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [0]:
from static_grader import grader

# PW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to use basic Python data structures, define functions, and control program flow. We will be using these concepts to perform some fundamental data wrangling tasks such as joining data sets together, splitting data into groups, and aggregating data into summary statistics.
**Please do not use `pandas` or `numpy` to answer these questions.**

We will be working with medical data from the British NHS on prescription drugs. Since this is real data, it contains many ambiguities that we will need to confront in our analysis. This is commonplace in data science, and is one of the lessons you will learn in this miniproject.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [0]:
%%bash
mkdir pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample.json.gz -nc -P ./pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/practices.json.gz -nc -P ./pw-data

--2019-06-11 08:40:33--  http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample.json.gz
Resolving dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)... 52.216.138.235
Connecting to dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)|52.216.138.235|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10367709 (9.9M) [application/json]
Saving to: ‘./pw-data/201701scripts_sample.json.gz’

     0K .......... .......... .......... .......... ..........  0% 24.4M 0s
    50K .......... .......... .......... .......... ..........  0% 24.3M 0s
   100K .......... .......... .......... .......... ..........  1% 35.1M 0s
   150K .......... .......... .......... .......... ..........  1% 34.9M 0s
   200K .......... .......... .......... .......... ..........  2% 32.2M 0s
   250K .......... .......... .......... .......... ..........  2% 34.6M 0s
   300K .......... .......... .......... .......... ..........  3% 41.3M 0s
   

## Loading the data

The first step of the project is to read in the data. We will discuss reading and writing various kinds of files later in the course, but the code below should get you started.

In [0]:
import gzip
import simplejson as json

In [0]:
with gzip.open('./pw-data/201701scripts_sample.json.gz', 'rb') as f:
    scripts = json.load(f)

with gzip.open('./pw-data/practices.json.gz', 'rb') as f:
    practices = json.load(f)

This data set comes from Britain's National Health Service. The `scripts` variable is a list of prescriptions issued by NHS doctors. Each prescription is represented by a dictionary with various data fields: `'practice'`, `'bnf_code'`, `'bnf_name'`, `'quantity'`, `'items'`, `'nic'`, and `'act_cost'`. 

In [0]:
scripts[:2]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500}]

A [glossary of terms](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10686/Download-glossary-of-terms-for-GP-prescribing---presentation-level/pdf/PLP_Presentation_Level_Glossary_April_2015.pdf/) and [FAQ](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10048/FAQs-Practice-Level-Prescribingpdf/pdf/PLP_FAQs_April_2015.pdf/) is available from the NHS regarding the data. Below we supply a data dictionary briefly describing what these fields mean.

| Data field |Description|
|:----------:|-----------|
|`'practice'`|Code designating the medical practice issuing the prescription|
|`'bnf_code'`|British National Formulary drug code|
|`'bnf_name'`|British National Formulary drug name|
|`'quantity'`|Number of capsules/quantity of liquid/grams of powder prescribed|
| `'items'`  |Number of refills (e.g. if `'quantity'` is 30 capsules, 3 `'items'` means 3 bottles of 30 capsules)|
|  `'nic'`   |Net ingredient cost|
|`'act_cost'`|Total cost including containers, fees, and discounts|

The `practices` variable is a list of member medical practices of the NHS. Each practice is represented by a dictionary containing identifying information for the medical practice. Most of the data fields are self-explanatory. Notice the values in the `'code'` field of `practices` match the values in the `'practice'` field of `scripts`.

In [0]:
practices[:2]

[{'code': 'A81001',
  'name': 'THE DENSHAM SURGERY',
  'addr_1': 'THE HEALTH CENTRE',
  'addr_2': 'LAWSON STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 1HU'},
 {'code': 'A81002',
  'name': 'QUEENS PARK MEDICAL CENTRE',
  'addr_1': 'QUEENS PARK MEDICAL CTR',
  'addr_2': 'FARRER STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 2AW'}]

In the following questions we will ask you to explore this data set. You may need to combine pieces of the data set together in order to answer some questions. Not every element of the data set will be used in answering the questions.

## Question 1: summary_statistics

Our beneficiary data (`scripts`) contains quantitative data on the number of items dispensed (`'items'`), the total quantity of item dispensed (`'quantity'`), the net cost of the ingredients (`'nic'`), and the actual cost to the patient (`'act_cost'`). Whenever working with a new data set, it can be useful to calculate summary statistics to develop a feeling for the volume and character of the data. This makes it easier to spot trends and significant features during further stages of analysis.

Calculate the sum, mean, standard deviation, and quartile statistics for each of these quantities. Format your results for each quantity as a list: `[sum, mean, standard deviation, 1st quartile, median, 3rd quartile]`. We'll create a `tuple` with these lists for each quantity as a final result.

In [0]:
import math
def describe(key):
    ctr=0
    l=[]
    total=0
    var=0
    for i in scripts:
        l.append(i[key])
        total+=i[key]
        ctr+=1
    avg = total/ctr
    for i in scripts:
        var+=(avg-i[key])**2
    s = (var/ctr)**0.5
    l.sort()
    med = l[math.floor(ctr/2)]
    q25 = l[math.floor(ctr/4)]
    q75 = l[math.floor(ctr*3/4)]

    return (total, avg, s, q25, med, q75)

In [0]:
summary = [('items', describe('items')),
           ('quantity', describe('quantity')),
           ('nic', describe('nic')),
           ('act_cost', describe('act_cost'))]

In [0]:
print (summary[0][1][0])

4410054


In [0]:
grader.score.pw__summary_statistics(summary)

Your score:  1.0


## Question 2: most_common_item

Often we are not interested only in how the data is distributed in our entire data set, but within particular groups -- for example, how many items of each drug (i.e. `'bnf_name'`) were prescribed? Calculate the total items prescribed for each `'bnf_name'`. What is the most commonly prescribed `'bnf_name'` in our data?

To calculate this, we first need to split our data set into groups corresponding with the different values of `'bnf_name'`. Then we can sum the number of items dispensed within in each group. Finally we can find the largest sum.

We'll use `'bnf_name'` to construct our groups. You should have *5619* unique values for `'bnf_name'`.

In [0]:
bnf_names = {}

for i in scripts:
    key=i['bnf_name']
    if key in bnf_names:
        bnf_names[key].append(i)
    else:
        bnf_names[key]=[i]
assert(len(bnf_names) == 5619)

We want to construct "groups" identified by `'bnf_name'`, where each group is a collection of prescriptions (i.e. dictionaries from `scripts`). We'll construct a dictionary called `groups`, using `bnf_names` as the keys. We'll represent a group with a `list`, since we can easily append new members to the group. To split our `scripts` into groups by `'bnf_name'`, we should iterate over `scripts`, appending prescription dictionaries to each group as we encounter them.

In [0]:
"""groups = {name: [] for name in bnf_names}
for script in scripts:
    groups[script['bnf_name']].append(script)  """
groups=bnf_names

Now that we've constructed our groups we should sum up `'items'` in each group and find the `'bnf_name'` with the largest sum. The result, `max_item`, should have the form `[(bnf_name, item total)]`, e.g. `[('Foobar', 2000)]`.

In [0]:
item=[]
for i in groups.keys():
    sum1=0
    for j in groups[i]:
        sum1+=j['items']
    item.append((sum1,i))
max_item = [max(item)[::-1]]

In [0]:
item[:10]

[(86, 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F'),
 (392, 'Alginate_Raft-Forming Oral Susp S/F'),
 (2636, 'Sod Algin/Pot Bicarb_Susp S/F'),
 (200, 'Sod Alginate/Pot Bicarb_Tab Chble 500mg'),
 (1978, 'Gaviscon Infant_Sach 2g (Dual Pack) S/F'),
 (5568, 'Gaviscon Advance_Liq (Aniseed) (Reckitt)'),
 (1078, 'Gaviscon Advance_Tab Chble Mint(Reckitt)'),
 (3443, 'Gaviscon Advance_Liq (Peppermint) S/F'),
 (3162, 'Peptac_Liq (Peppermint) S/F'),
 (776, 'Alverine Cit_Cap 60mg')]

**TIP:** If you are getting an error from the grader below, please make sure your answer conforms to the correct format of `[(bnf_name, item total)]`.

In [0]:
grader.score.pw__most_common_item(max_item)

Your score:  1.0


**Challenge:** Write a function that constructs groups as we did above. The function should accept a list of dictionaries (e.g. `scripts` or `practices`) and a tuple of fields to `groupby` (e.g. `('bnf_name')` or `('bnf_name', 'post_code')`) and returns a dictionary of groups. The following questions will require you to aggregate data in groups, so this could be a useful function for the rest of the miniproject.

In [0]:
def group_by_field(data, fields):
    groups_dic={}
    for field in fields:
        groups = {}
        for i in data:
            key=i['%s'%(field)]
            if key in groups:
                groups[key].append(i)
            else:
                groups[key]=[i]
        groups_dic['%s'%(field)]=groups
    return groups_dic

In [0]:
groups = group_by_field(scripts, ('bnf_name',))
item=[]
for k in groups.keys():
    for i in groups[k].keys():
        sum1=0
        for j in groups[k][i]:
            sum1+=j['items']
        item.append((sum1,i))
    test_max_item = [max(item)[::-1]]


assert test_max_item == max_item

## Question 3: postal_totals

Our data set is broken up among different files. This is typical for tabular data to reduce redundancy. Each table typically contains data about a particular type of event, processes, or physical object. Data on prescriptions and medical practices are in separate files in our case. If we want to find the total items prescribed in each postal code, we will have to _join_ our prescription data (`scripts`) to our clinic data (`practices`).

Find the total items prescribed in each postal code, representing the results as a list of tuples `(post code, total items prescribed)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes. Only include post codes if there is at least one prescription from a practice in that post code.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code.

We can join `scripts` and `practices` based on the fact that `'practice'` in `scripts` matches `'code'` in `practices'`. However, we must first deal with the repeated values of `'code'` in `practices`. We want the alphabetically first postal codes.

In [0]:
practice_postal = {}
for practice in practices:
    if practice['code'] in practice_postal:
        if (practice_postal[practice['code']]>practice['post_code']):
            practice_postal[practice['code']]=practice['post_code']
    else:
        practice_postal[practice['code']] = practice['post_code']

**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [0]:
assert practice_postal['K82019'] == 'HP21 8TR'

**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [0]:
assert practice_postal['K82019'] == 'HP21 8TR'

Now we can join `practice_postal` to `scripts`.

In [0]:
joined = scripts[:]
for script in joined:
    if script['practice'] in practice_postal.keys():
        script['post_code'] = practice_postal[script['practice']]

Finally we'll group the prescription dictionaries in `joined` by `'post_code'` and sum up the items prescribed in each group, as we did in the previous question.

In [0]:
items_by_post = group_by_field(scripts, ('post_code',))
items_by_post_1 ={}
for i in scripts:
    key=i['post_code']
    if key in items_by_post_1:
        items_by_post_1[key].append(i)
    else:
        items_by_post_1[key]=[i]
groups=items_by_post_1        
postal_total1=[]
for i in groups.keys():
    sum1=0
    for j in groups[i]:
        sum1+=j['items']
    if (sum1)!=0:
        postal_total1.append((i,sum1))

In [0]:
practice_postal = {}
for practice in practices:
    if practice['code'] in practice_postal:
        if (practice_postal[practice['code']]>practice['post_code']):
            practice_postal[practice['code']]=practice['post_code']
    else:
        practice_postal[practice['code']] = practice['post_code']

joined = scripts[:]
for script in joined:
    if script['practice'] in practice_postal.keys():
        script['post_code'] = practice_postal[script['practice']]

prescriptionByPostCode = {}
for element in joined:
    if element['post_code'] not in prescriptionByPostCode:
       prescriptionByPostCode[element['post_code']] = [element['items']]
    else:
        prescriptionByPostCode[element['post_code']].append(element['items'])

items_by_post = [sum(prescriptionByPostCode[x]) for x in prescriptionByPostCode]
keys = list(prescriptionByPostCode.keys())
pairs = list(zip(keys, items_by_post))
pairs.sort()
items_by_post = []
for pair in pairs:
    if pair[1] > 0:
        items_by_post.append(pair)            

In [0]:
grader.score.pw__postal_totals(items_by_post[:100])

Your score:  1.0


In [0]:
(postal_total1).sort()
postal_sorted=(postal_total1)[:100]
#print (postal_sorted[:100][::-1])
#postal_total = [(j,i) for i,j in (postal_sorted[::-1][:100])]

grader.score.pw__postal_totals(postal_sorted)

Your score:  1.0


In [0]:
print(items_by_post[:100])
print(postal_sorted)
sum1=0
for i in postal_sorted:
    sum1+=i[1]
print (sum1)

[('B11 4BW', 20673), ('B18 7AL', 19001), ('B21 9RY', 29103), ('B23 6DJ', 24859), ('B70 7AW', 36531), ('BB11 2DL', 34356), ('BB2 1AX', 28254), ('BB3 1PY', 54514), ('BB4 5SL', 29388), ('BB7 2JG', 44585), ('BB8 0JZ', 54380), ('BB9 7SR', 38224), ('BD3 8QH', 21010), ('BH18 8EE', 39413), ('BH23 3AF', 32545), ('BL1 8TU', 26132), ('BL3 5HP', 27147), ('BL9 0NJ', 32062), ('BL9 0SN', 35275), ('CB9 8HF', 51337), ('CH1 4DS', 34915), ('CH65 6TG', 25090), ('CT11 8AD', 44358), ('CV1 4FS', 37210), ('CW1 3AW', 64104), ('CW5 5NX', 38797), ('CW7 1AT', 43164), ('DA1 2HA', 26075), ('DA11 8BZ', 24090), ('DN22 7XF', 43091), ('DN34 4GB', 48013), ('FY2 0JG', 69118), ('FY4 1TJ', 62886), ('FY5 2TZ', 44258), ('FY7 8GU', 34473), ('GL1 3PX', 38120), ('GL50 4DP', 74822), ('GU9 9QS', 32131), ('HA0 4UZ', 22755), ('HA3 7LT', 32113), ('HG1 5AR', 32684), ('HU7 4DW', 49107), ('KT14 6DH', 26758), ('KT6 6EZ', 38975), ('L31 0DJ', 32065), ('L36 7XY', 22965), ('L5 0QW', 24676), ('L7 6HD', 42569), ('LA1 1PN', 47335), ('LE10 1DS'

## Question 4: items_by_region

Now we'll combine the techniques we've developed to answer a more complex question. Find the most commonly dispensed item in each postal code, representing the results as a list of tuples (`post_code`, `bnf_name`, amount dispensed as proportion of total). Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** We'll continue to use the `joined` variable we created before, where we've chosen the alphabetically first postal code for each practice. Additionally, some postal codes will have multiple `'bnf_name'` with the same number of items prescribed for the maximum. In this case, we'll take the alphabetically first `'bnf_name'`.

Now we need to calculate the total items of each `'bnf_name'` prescribed in each `'post_code'`. Use the techniques we developed in the previous questions to calculate these totals. You should have 141196 `('post_code', 'bnf_name')` groups.

In [0]:
joined[:2]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000,
  'post_code': 'SK11 6JL'},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500,
  'post_code': 'SK11 6JL'}]

In [0]:
bnf_names = {}
for i in joined:
    key=i['bnf_name']
    key2=i['post_code']
    if (key,key2) in bnf_names:
        bnf_names[(key,key2)].append(i)
    else:
        bnf_names[(key,key2)]=[i]
groups=bnf_names
item=[]
for i,j in groups.keys():
    sum1=0
    for k in groups[(i,j)]:
        sum1+=k['items']
    item.append([j,i,sum1])

In [0]:
total_items_by_bnf_post = item
assert len(total_items_by_bnf_post) == 141196

Let's use `total_by_item_post` to find the maximum item total for each postal code. To do this, we will want to regroup `total_by_item_post` by `'post_code'` only, not by `('post_code', 'bnf_name')`. First let's turn `total_by_item_post` into a list of dictionaries (similar to `scripts` or `practices`) and then group it by `'post_code'`. You should have 118 groups in `total_by_item_post` after grouping it by `'post_code'`.

In [0]:
def group_by_field2(data, fields):
    groups_dic={}
    for field in fields:
        groups = {}
        for i in data:
            key=i['%s'%(field)]
            if key in groups.keys():
                flag=0
                for name in range(len(groups[key])):
                    if (groups[key][name][1]==i['bnf_name']):
                        groups[key][name][0]+=i['items']
                        flag=1
                        break
                if flag==0:
                    groups[key].append([i['items'],i['bnf_name']])
            else:
                groups[key]=[[i['items'],i['bnf_name']]]
        groups_dic['%s'%(field)]=groups
        
    return groups_dic

In [0]:
total_items_by_post = group_by_field2(joined, ('post_code',))['post_code']
assert len(total_items_by_post) == 118

Now we will aggregate the groups in `total_by_item_post` to create `max_item_by_post`. Some `'bnf_name'` have the same item total within a given postal code. Therefore, if more than one `'bnf_name'` has the maximum item total in a given postal code, we'll take the alphabetically first `'bnf_name'`. We can do this by [sorting](https://docs.python.org/2.7/howto/sorting.html) each group according to the item total and `'bnf_name'`.

In [0]:
test=total_items_by_post

In [0]:
for key in test:
    test['%s'%key].sort(reverse=True)

In [0]:
anslist=[]
ctr=0
total_items=items_by_post
for key in sorted(list(test.keys())):
    anslist.append((key,'%s'%max(test['%s'%key])[1],(max(test['%s'%key])[0]/total_items[ctr][1])))
    ctr+=1
anslist.sort()
#anslist.append((key,test['%s'%key],max(test['%s'%key][])))----//total_items[ctr][1]

In [0]:
list(test.keys()).sort()

In [0]:
max_item_by_post = anslist[:100]

In order to express the item totals as a proportion of the total amount of items prescribed across all `'bnf_name'` in a postal code, we'll need to use the total items prescribed that previously calculated as `items_by_post`. Calculate the proportions for the most common `'bnf_names'` for each postal code. Format your answer as a list of tuples: `[(post_code, bnf_name, total)]`

In [0]:
items_by_region = [('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.0341508247)] * 100

In [0]:
items_by_region = anslist[:100]

In [0]:
grader.score.pw__items_by_region(items_by_region)

Your score:  1.0


*Copyright &copy; 2017 The Data Incubator.  All rights reserved.*

# DW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to wrangle tabular data set and aggregate large data sets into meaningful summary statistics. We will be working with the same medical data used in the `pw` miniproject, but will be leveraging the power of Pandas to more efficiently represent and act on our data.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [0]:
#!mkdir dw-data
#!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201701scripts_sample.csv.gz -nc -P ./dw-data/
#!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201606scripts_sample.csv.gz -nc -P ./dw-data/
#!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/practices.csv.gz -nc -P ./dw-data/
#!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/chem.csv.gz -nc -P ./dw-data/

## Loading the data

Similar to the `PW` miniproject, the first step is to read in the data. The data files are stored as compressed CSV files. You can load the data into a Pandas DataFrame by making use of the `gzip` package to decompress the files and Panda's `read_csv` methods to parse the data into a DataFrame. You may want to check the Pandas documentation for parsing [CSV](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) files for reference.

For a description of the data set please, refer to the [PW miniproject](./pw.ipynb). **Note that all questions make use of the 2017 data only, except for Question 5 which makes use of both the 2017 and 2016 data.**

In [0]:
import pandas as pd
import numpy as np
import gzip

In [0]:
!ls dw-data

201606scripts_sample.csv.gz  chem.csv.gz
201701scripts_sample.csv.gz  practices.csv.gz


In [0]:
# load the 2017 data
with gzip.open('./dw-data/201701scripts_sample.csv.gz','r') as f2017:
    scripts=pd.read_csv(f2017)
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [0]:
col_names=[ 'code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']
with gzip.open('./dw-data/practices.csv.gz','r') as f_practices:
    practices =pd.read_csv(f_practices,names=col_names)
practices.head()

,code,name,addr_1,addr_2,borough,village,post_code
0,A81001,THE DENSHAM SURGERY,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,TS18 1HU
1,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
2,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
3,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,NaN,MIDDLESBROUGH,CLEVELAND,TS1 3BE
4,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ


In [0]:
with gzip.open('./dw-data/chem.csv.gz','r') as f_chem:
    chem =pd.read_csv(f_chem)
chem.head()

,CHEM SUB,NAME
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


Now that we've loaded in the data, let's first replicate our results from the `PW` miniproject. Note that we are now working with a larger data set so the answers will be different than in the `PW` miniproject even if the analysis is the same.

## Question 1: summary_statistics

In the `PW` miniproject we first calculated the total, mean, standard deviation, and quartile statistics of the `'items'`, `'quantity'`', `'nic'`, and `'act_cost'` fields. To do this we had to write some functions to calculate the statistics and apply the functions to our data structure. The DataFrame has a `describe` method that will calculate most (not all) of these things for us.

Submit the summary statistics to the grader as a list of tuples: [('act_cost', (total, mean, std, q25, median, q75)), ...]

In [0]:
def describe(head):
    l=(np.array([i for i in scripts['%s'%head].describe()]).tolist()[1:7])
    l.insert(0,scripts['%s'%head].sum())
    l.pop(3)
    return tuple(l)

In [0]:
describe('items')

(8888304, 9.133135976111625, 29.204198282803603, 1.0, 2.0, 6.0)

In [0]:
scripts['items'].describe()

count    973193.000000
mean          9.133136
std          29.204198
min           1.000000
25%           1.000000
50%           2.000000
75%           6.000000
max        2384.000000
Name: items, dtype: float64

In [0]:
summary_stats = [('items', describe('items')), ('quantity', describe('quantity')), ('nic', describe('nic')), ('act_cost', describe('act_cost'))]

In [0]:
grader.score.dw__summary_statistics(summary_stats)

Your score:  1.0


## Question 2: most_common_item

We can also easily compute summary statistics on groups within the data. In the `pw` miniproject we had to explicitly construct the groups based on the values of a particular field. Pandas will handle that for us via the `groupby` method. This process is [detailed in the Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html).

Use `groupby` to calculate the total number of items dispensed for each `'bnf_name'`. Find the item with the highest total and return the result as `[(bnf_name, total)]`.

In [0]:
most_common_item = [(scripts.groupby('bnf_name')['items'].sum().argmax(),scripts.groupby('bnf_name')['items'].sum().max())]

In [0]:
scripts.groupby('bnf_name')['items'].sum().sort_values(ascending=False).head(1)

bnf_name
Omeprazole_Cap E/C 20mg    218583
Name: items, dtype: int64

In [0]:
grader.score.dw__most_common_item(most_common_item)

Error!
There was an error grading this question. Please notify a TDI staff member.


## Question 3: items_by_region

Now let's find the most common item by post code. The post code information is in the `practices` DataFrame, and we'll need to `merge` it into the `scripts` DataFrame. Pandas provides [extensive documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html) with diagrammed examples on different methods and approaches for joining data. The `merge` method is only one of many possible options.

Return your results as a list of tuples `(post code, item name, amount dispensed as % of total)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code. Note some postal codes may have multiple `'bnf_name'` with the same prescription rate for the maximum. In this case, take the alphabetically first `'bnf_name'` (as in the PW miniproject).

In [0]:
sort_practices=practices.sort_values(by=['code','post_code']).drop_duplicates(subset='code',keep='first')
#.set_index('code')
sort_practices=sort_practices.rename(index=str, columns={"code": "practice"})

In [0]:
sort_practices.head(2)

,practice,name,addr_1,addr_2,borough,village,post_code
0,A81001,THE DENSHAM SURGERY,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,TS18 1HU
1,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW


In [0]:
s=scripts
merge_df=s.merge(sort_practices, on='practice')

In [0]:
merge_df.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,name,addr_1,addr_2,borough,village,post_code
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF


In [0]:
practices['post_code'].nunique()

8306

In [0]:
test=merge_df.groupby(['post_code','bnf_name']).agg({'items':'sum'})
test

items
post_code bnf_name                                       
B11 4BW   3m Health Care_Cavilon Durable Barrier C      7
          3m Health Care_Cavilon No Sting Barrier       2
          Abasaglar KwikPen_100u/ml 3ml Pf Pen          2
          Abidec_Dps                                   63
          Able Spacer + Sml/Med Mask                    1
          Accrete D3_Tab                               24
          Acetazolamide_Tab 250mg                       2
          Acetic Acid_Ear Spy 2% 5ml                    9
          Acetylcy_Tab 600mg                            2
          Aciclovir_Crm 5%                             13
          Aciclovir_Oral Susp 400mg/5ml S/F             1
          Aciclovir_Tab 200mg                           4
          Aciclovir_Tab 800mg                           9
          Aciclovir_Tab Disper 200mg                    1
          Aciclovir_Tab Disper 400mg                    2
          Aciferol D3_Cap 50 000u                       2
          Aclidinium Brom_Pdr For Inh 375mcg (60D)      4
          Acnecide_Aq Gel 5%                            1
          ActiLymph Class 1 B/Knee Closed Toe Blk       1
          Active (Reagent)_Strips                       1
          Adalat LA 20_Tab 20mg                         2
          Adalat LA 30_Tab 30mg                         3
          Adalat LA 60_Tab 60mg                        11
          Adapalene/Benzoyl Per_Gel 0.1%/2.5%           3
          Adapalene_Gel 0.1%                            1
          Adcal-D3_Capl 750mg/200u                     48
          Adcal-D3_Dissolve Tab Eff 1.5g/400u(Lem)      8
          Adcal-D3_Tab Chble (Lem)                      7
          Adcal-D3_Tab Chble (Tutti-Frutti)            48
          Adcal_Tab Chble 1.5g                          3
...                                                   ...
YO16 4LZ  Zocor_Tab 20mg                                1
          Zoely_Tab 2.5mg/1.5mg                         2
          Zoladex LA_Implant 10.8mg SafeSystem Pfs     10
          Zoladex_Implant 3.6mg SafeSystem Pfs          3
          Zolmitriptan_Nsl Spy 5mg/0.1ml Ud             1
          Zolmitriptan_Tab 2.5mg                        7
          Zolpidem Tart_Tab 10mg                        5
          Zolpidem Tart_Tab 5mg                         4
          Zolvera_Oral Soln 40mg/5ml S/F                1
          Zomig_Nsl Spy 5mg/0.1ml Ud                    1
          Zomorph_Cap 100mg                             7
          Zomorph_Cap 10mg                             57
          Zomorph_Cap 30mg                             33
          Zomorph_Cap 60mg                             10
          Zonegran_Cap 100mg                            1
          Zonegran_Cap 25mg                             1
          Zonisamide_Cap 100mg                          2
          Zopiclone_Tab 3.75mg                         79
          Zopiclone_Tab 7.5mg                          86
          Zoton_FasTab Tab 15mg                         1
          Zoton_FasTab Tab 30mg                         2
          Zuclopenth Decan_Inj 500mg/ml 1ml Amp         1
          Zuclopenth HCl_Tab 25mg                       6
          Zumenon_Tab 2mg                               1
          Zydol SR_Tab 200mg                            1
          Zydol SR_Tab 50mg                             1
          easiGRIP 12cm x 1m Stkntte Elasctd Tublr      1
          iMEDicare SomaCorrect Xtra Vacuum Pump        1
          iMEDicare_Urine Collection Bag 500ml          1
          kliniderm superabsorbent 20cm x 30cm Pfa      1

[579197 rows x 1 columns]

In [0]:
total_items_by_post=merge_df.groupby('post_code')['items'].sum().to_frame('Max').reset_index()

In [0]:
max_items_by_post=test.reset_index().sort_values(by=['items','bnf_name'],ascending=False).drop_duplicates(subset='post_code',keep='first').sort_values(by='post_code')

In [0]:
items_by_post=max_items_by_post.merge(total_items_by_post, on='post_code')

In [0]:
items_by_post['proportion']=items_by_post['items']/items_by_post['Max']

In [0]:
items_by_region = list(items_by_post.drop(labels=['items','Max'], axis=1).itertuples(index=False))[:100]

In [0]:
grader.score.dw__items_by_region(items_by_region)

Your score:  1.0


## Question 4: script_anomalies

Drug abuse is a source of human and monetary costs in health care. A first step in identifying practitioners that enable drug abuse is to look for practices where commonly abused drugs are prescribed unusually often. Let's try to find practices that prescribe an unusually high amount of opioids. The opioids we'll look for are given in the list below.

In [0]:
opioids = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine']

These are generic names for drugs, not brand names. Generic drug names can be found using the `'bnf_code'` field in `scripts` along with the `chem` table.. Use the list of opioids provided above along with these fields to make a new field in the `scripts` data that flags whether the row corresponds with a opioid prescription.

In [0]:
#Single line/Inline implementation
#bnf_name=(pd.DataFrame(scripts[['bnf_name','bnf_code']])).sort_values(by=['bnf_name']).drop_duplicates(subset=['bnf_name'],keep='first').reset_index().drop(labels=['index'],axis=1)

In [0]:
chem.head()

,CHEM SUB,NAME
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


In [0]:
import re
def is_opioid(attr):
    for x in opioids:
        if re.search(x.lower(), attr.lower()):
            return True

In [0]:
chem['is_opioids']=chem['NAME'].apply(is_opioid).fillna(False)

In [0]:
new_scripts=scripts.merge(chem.drop_duplicates(subset=['CHEM SUB'], keep='first'),left_on='bnf_code',right_on='CHEM SUB',how='left').drop(labels='NAME',axis=1)
new_scripts['is_opioids'].fillna(False, inplace=True)
new_scripts[new_scripts['is_opioids']].shape

(34843, 9)

In [0]:
chem[chem['is_opioids']]

,CHEM SUB,NAME,is_opioids
88,0104020D0,Codeine Phosphate Compound Mixtures,True
91,0104020N0,Opium & Morphine,True
691,0309010C0,Codeine Phosphate,True
693,0309010N0,Diamorphine Hydrochloride,True
695,0309010S0,Methadone Hydrochloride,True
701,0309020AC,Guaiacol & Codeine,True
955,0407010F0,Co-Codamol (Codeine Phos/Paracetamol),True
959,0407010M0,Co-Codaprin (Codeine Phos/Aspirin),True
960,0407010N0,Co-Dydramol (Dihydrocodeine/Paracet),True
963,0407010R0,Aspirin Phenacetin & Codeine(Codeine Co),True


In [0]:
drug_Names_df = scripts.join(chem.drop_duplicates(subset='CHEM SUB',keep='first').set_index('CHEM SUB'),on='bnf_code', how='left', )
my_string = "|".join(opioids)
my_string = my_string[1:] 
opioids_attr = drug_Names_df['NAME'].str.contains(my_string, regex=True, case=False) 
drug_Names_df['opioids'] = opioids_attr 
drug_Names_df['opioids'].fillna(False, inplace = True) 
drug_Names_df['opioids'].mean()

0.03580276471367961

In [0]:
drug_Names_df[drug_Names_df['opioids']].shape

(34843, 9)

In [0]:
new_scripts.shape

(974010, 9)

In [0]:
bnf_name[bnf_name['bnf_name'].str.contains('codeine')]

In [0]:
bnf_name[bnf_name['bnf_name'].str.contains('Cyclizine')]

,bnf_name,bnf_code,opioids
3399,Cyclizine HCl_Oral Soln 50mg/5ml,0406000F0,False
3400,Cyclizine HCl_Oral Susp 50mg/5ml,0406000F0,False
3401,Cyclizine HCl_Tab 50mg,0406000F0,False
3402,Cyclizine Lact_Inj 50mg/ml 1ml Amp,0406000G0,False
4017,Dipipanone HCl/Cyclizine HCl_Tab 10/30mg,0407020H0,False


In [0]:
new_scripts[new_scripts['is_opioids']].count()

practice      34843
bnf_code      34843
bnf_name      34843
items         34843
nic           34843
act_cost      34843
quantity      34843
CHEM SUB      34843
is_opioids    34843
dtype: int64

In [0]:
new_scripts.head(1)

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,CHEM SUB,is_opioids
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,0106020C0,False


In [0]:
overall_is_opioids=new_scripts['is_opioids'].mean()
overall_is_opioids

0.03580276471367961

Now for each practice calculate the proportion of its prescriptions containing opioids.

**Hint:** Consider the following list: `[0, 1, 1, 0, 0, 0]`. What proportion of the entries are 1s? What is the mean value?

In [0]:
opioids_per_practice = new_scripts.groupby('practice')['is_opioids'].mean()

In [0]:
opioids_per_practice['Y01852']

0.8571428571428571

How do these proportions compare to the overall opioid prescription rate? Subtract off the proportion of all prescriptions that are opioids from each practice's proportion.

In [0]:
relative_opioids_per_practice = opioids_per_practice-overall_is_opioids

Now that we know the difference between each practice's opioid prescription rate and the overall rate, we can identify which practices prescribe opioids at above average or below average rates. However, are the differences from the overall rate important or just random deviations? In other words, are the differences from the overall rate big or small?

To answer this question we have to quantify the difference we would typically expect between a given practice's opioid prescription rate and the overall rate. This quantity is called the **standard error**, and is related to the **standard deviation**, $\sigma$. The standard error in this case is

$$ \frac{\sigma}{\sqrt{n}} $$

where $n$ is the number of prescriptions each practice made. Calculate the standard error for each practice. Then divide `relative_opioids_per_practice` by the standard errors. We'll call the final result `opioid_scores`.

In [0]:
overall_pres=scripts.groupby('practice')['bnf_code'].count()
overall_press_by_practice=pd.DataFrame(overall_pres).rename(columns={'bnf_code':'total_pres'}).reset_index()
overall_press_by_practice.head(5)

,practice,total_pres
0,A81005,1507
1,A81007,1454
2,A81011,1568
3,A81012,1332
4,A81017,2150


In [0]:
new_scripts['is_opioids'].std()

0.18579817605238425

In [0]:
standard_error_per_practice = new_scripts['is_opioids'].std()/np.sqrt(new_scripts.groupby('practice')['is_opioids'].count())
opioid_scores = relative_opioids_per_practice/standard_error_per_practice

The quantity we have calculated in `opioid_scores` is called a **z-score**:

$$ \frac{\bar{X} - \mu}{\sqrt{\sigma^2/n}} $$

Here $\bar{X}$ corresponds with the proportion for each practice, $\mu$ corresponds with the proportion across all practices, $\sigma^2$ corresponds with the variance of the proportion across all practices, and $n$ is the number of prescriptions made by each practice. Notice $\bar{X}$ and $n$ will be different for each practice, while $\mu$ and $\sigma$ are determined across all prescriptions, and so are the same for every z-score. The z-score is a useful statistical tool used for hypothesis testing, finding outliers, and comparing data about different types of objects or events.

Now that we've calculated this statistic, take the 100 practices with the largest z-score. Return your result as a list of tuples in the form `(practice_name, z-score, number_of_scripts)`. Sort your tuples by z-score in descending order. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [0]:
merge_practice=practices.drop(labels=['addr_1','addr_2','borough','village','post_code'], axis=1).rename(columns={'code':'practice'}).sort_values(by=['name','practice']).drop_duplicates(subset=['practice'],keep='first')

,practice,name
7012,P87657,(IRLAM) SALFORD CARE CTRS MEDICAL PRACTI
10922,Y05381,0-19 EAST CHESHIRE HEALTH VISITORS
11984,Y04082,0-19 PUBLIC HEALTH SERVICE HARTLEPOOL
7573,Y00364,1 TO 1 CENTRE
6836,P87652,1/LOWER BROUGHTON MEDICAL PRACTICE


In [0]:
ans_df=pd.DataFrame(opioid_scores).reset_index().rename(columns={'is_opioids':'z-score'})

In [0]:
ans_df.head(6)

,practice,z-score,name,total_pres
714,Y01852,11.695818,NATIONAL ENHANCED SERVICE,7
754,Y03006,7.339043,OUTREACH SERVICE NH / RH,2
772,Y03668,6.150582,BRISDOC HEALTHCARE SERVICES OOH,60
261,G81703,5.123032,H&R P C SPECIAL SCHEME,36
819,Y04997,4.958866,HMR BARDOC OOH,321
799,Y04585,4.888878,INTEGRATED CARE 24 LTD (CWSX OOH),426


In [0]:
ans_df=ans_df[['name','z-score','total_pres']]

In [0]:
ans_df.head()

,name,z-score,total_pres
714,NATIONAL ENHANCED SERVICE,11.695818,7
754,OUTREACH SERVICE NH / RH,7.339043,2
772,BRISDOC HEALTHCARE SERVICES OOH,6.150582,60
261,H&R P C SPECIAL SCHEME,5.123032,36
819,HMR BARDOC OOH,4.958866,321


In [0]:
unique_practices = pd.DataFrame(opioid_scores).reset_index().rename(columns={'is_opioids':'z-score'}).merge(merge_practice, on='practice').merge(overall_press_by_practice,on='practice').sort_values(by=['z-score','name'],ascending=False).drop_duplicates(subset='practice', keep='first')
anomalies = list(unique_practices[['name','z-score','total_pres']].itertuples(index=False))[:100]

In [0]:
grader.score.dw__script_anomalies(anomalies)

Your score:  1.0


## Question 5: script_growth

Another way to identify anomalies is by comparing current data to historical data. In the case of identifying sites of drug abuse, we might compare a practice's current rate of opioid prescription to their rate 5 or 10 years ago. Unless the nature of the practice has changed, the profile of drugs they prescribe should be relatively stable. We might also want to identify trends through time for business reasons, identifying drugs that are gaining market share. That's what we'll do in this question.

We'll load in beneficiary data from 6 months earlier, June 2016, and calculate the percent growth in prescription rate from June 2016 to January 2017 for each `bnf_name`. We'll return the 50 items with largest growth and the 50 items with the largest shrinkage (i.e. negative percent growth) as a list of tuples sorted by growth rate in descending order in the format `(script_name, growth_rate, raw_2016_count)`. You'll notice that many of the 50 fastest growing items have low counts of prescriptions in 2016. Filter out any items that were prescribed less than 50 times.

In [0]:
with gzip.open('./dw-data/201606scripts_sample.csv.gz','r') as f2016:
    scripts16=pd.read_csv(f2016)
scripts16.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85638,0301011R0,Salamol_Inha 100mcg (200 D) CFF (Teva),2,2.92,2.73,2
1,N85638,0301011R0,Easyhaler_Salbutamol Sulf 200mcg (200D),1,6.63,6.15,1
2,N85638,0301020I0,Ipratrop Brom_Inh Soln 500mcg/2ml Ud,1,1.77,1.75,12
3,N85638,0301020I0,Ipratrop Brom_Inh Soln 250mcg/1ml Ud,1,4.47,4.15,20
4,N85638,0302000C0,Clenil Modulite_Inha 50mcg (200D),1,3.70,3.44,1


In [0]:
script16_grouped=pd.DataFrame(scripts16.groupby('bnf_name')['items'].count()).reset_index()
script16_grouped['del']=script16_grouped['items']<50
script16_grouped=script16_grouped[~script16_grouped['del']].drop(labels='del',axis=1)

In [0]:
script17_grouped=pd.DataFrame(scripts.groupby('bnf_name')['items'].count()).reset_index()
#script17_grouped['del']=script17_grouped['items']<50
#script17_grouped=script17_grouped[~script17_grouped['del']].drop(labels='del',axis=1)

In [0]:
merged=script16_grouped.merge(script17_grouped, on='bnf_name')
merged['growth']=(merged['items_y']-merged['items_x'])/merged['items_x']
merged.sort_values(by=['growth'],ascending=False, inplace=True)
merged.reset_index(inplace=True)
merged.drop(labels='index',axis=1,inplace=True)

In [0]:
script_growth=list(merged[['bnf_name','growth','items_x']].itertuples(index=False))[:50]+list(merged[['bnf_name','growth','items_x']].itertuples(index=False))[::-1][:50][::-1]

In [0]:
grader.score.dw__script_growth(script_growth)

Your score:  1.0


## Question 6: rare_scripts

Does a practice's prescription costs originate from routine care or from reliance on rarely prescribed treatments? Commonplace treatments can carry lower costs than rare treatments because of efficiencies in large-scale production. While some specialist practices can't help but avoid prescribing rare medicines because there are no alternatives, some practices may be prescribing a unnecessary amount of brand-name products when generics are available. Let's identify practices whose costs disproportionately originate from rarely prescribed items.

First we have to identify which `'bnf_code'` are rare. To do this, find the probability $p$ of a prescription having a particular `'bnf_code'` if the `'bnf_code'` was randomly chosen from the unique options in the beneficiary data. We will call a `'bnf_code'` rare if it is prescribed at a rate less than $0.1p$.

In [0]:
p = 1/(scripts['bnf_code'].nunique())
rates = pd.DataFrame(scripts.groupby('bnf_code')['bnf_name'].count()/scripts['bnf_name'].count()).reset_index()
rates.rename(index=str, columns={'bnf_name':'rate'},inplace=True)
rates['is_rare'] = rates['rate']<0.1*p

In [0]:
merged_scripts=scripts.merge(rates, on='bnf_code')


In [0]:
merged_scripts[merged_scripts['is_rare']].nunique()

practice     766
bnf_code     844
bnf_name    2237
items         35
nic         4087
act_cost    4747
quantity     309
rate          49
is_rare        1
dtype: int64

In [0]:
scripts.head(2)

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30


In [0]:
rates.head(2)

,bnf_code,rate,is_rare
0,0101010C0,0.000035,True
1,0101010F0,0.000002,True


In [0]:
merged_scripts.head(2)

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,rate,is_rare
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,0.001064,False
1,N81013,0106020C0,Bisacodyl_Tab E/C 5mg,19,28.10,28.14,860,0.001064,False


Now for each practice, calculate the proportion of costs that originate from prescription of rare treatments (i.e. rare `'bnf_code'`). Use the `'act_cost'` field for this calculation.

In [0]:
total_act=merged_scripts.groupby('practice')['act_cost'].sum()
rare_act=merged_scripts.groupby(['practice','is_rare'])['act_cost'].agg({'rare_cost':'sum'})
rare_act.reset_index(inplace=True)
rare_act=rare_act[rare_act['is_rare']].set_index('practice')

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


In [0]:
rare_cost_prop=rare_act.join(total_act).drop(labels='is_rare', axis=1)
rare_cost_prop['rare_cost_prop']=rare_cost_prop['rare_cost']/rare_cost_prop['act_cost']

In [0]:
rare_cost_prop.head()

,rare_cost,act_cost,rare_cost_prop
practice,,,
A81005,1247.83,103840.82,0.012017
A81007,951.06,113482.49,0.008381
A81011,816.02,159507.03,0.005116
A81012,1145.11,83296.81,0.013747
A81017,1712.15,232656.17,0.007359


Now we will calculate a z-score for each practice based on this proportion.
First take the difference of `rare_cost_prop` and the proportion of costs originating from rare treatments across all practices.

In [0]:
rare_cost_prop['all_prac_prop']=rare_cost_prop['rare_cost'].sum()/rare_cost_prop['act_cost'].sum()

In [0]:
rare_cost_prop['prop_diff']=rare_cost_prop['rare_cost_prop']-rare_cost_prop['all_prac_prop']

Now we will estimate the standard errors (i.e. the denominator of the z-score) by simply taking the standard deviation of this difference.

In [0]:
standard_errors = rare_cost_prop['prop_diff'].std()

Finally compute the z-scores. Return the practices with the top 100 z-scores in the form `(post_code, practice_name, z-score)`. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [0]:
rare_cost_prop['z-score']=rare_cost_prop['prop_diff']/standard_errors

In [0]:
rare_cost_prop.sort_values(by='z-score',ascending=False, inplace=True)

In [0]:
merge_practice=practices.drop(labels=['addr_1','addr_2','borough','village','post_code'], axis=1).rename(columns={'code':'practice'}).sort_values(by=['name','practice']).drop_duplicates(subset=['practice'],keep='first')

In [0]:
rare_cost_prop=rare_cost_prop.reset_index().merge(merge_practice,on='practice').sort_values(by='z-score',ascending=False)

In [0]:
rare_scripts = list(rare_cost_prop[['practice','name','z-score']].itertuples(index=False))[:100]

In [0]:
grader.score.dw__rare_scripts(rare_scripts)

Your score:  1.0


*Copyright &copy; 2019 The Data Incubator.  All rights reserved.*